# Notebook 2: News Classification

This notebook implements:
- Loading pretrained multilingual models (XLM-RoBERTa)
- Fine-tuning for Nepali news classification
- Training on 13+ news categories
- Model evaluation and saving
- Prediction on new articles

In [2]:
%pip install numpy pandas matplotlib seaborn scikit-learn torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 17.0 MB/s  0:00:01m0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 18.1 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 24.2 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 26.9 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 14.7 MB/s  0:00:25m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 20.6 MB/s  0:00:16m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 31.8 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 38.5 MB/s  0:00:02m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 33.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 17.8 MB/s  0:00:20m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 38.3 MB/s  0:00:05m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1

In [3]:
%pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 35.1 MB/s  0:00:01m0:00:0100:01
  Attempting uninstall: fsspec━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  1/14 [pyarrow]
    Found existing installation: fsspec 2026.1.0━━━━━━━━━━━━━━  1/14 [pyarrow]
    Uninstalling fsspec-2026.1.0:━━━━━━━━━━━━━━━━━━━━━━━━━━━━  4/14 [fsspec]
      Successfully uninstalled fsspec-2026.1.0━━━━━━━━━━━━━━━━  4/14 [fsspec]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━ 12/14 [datasets]ess]  WARNING: The script datasets-cli is installed in '/usr/local/python/3.12.1/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━ 13/14 [evaluate]  WARNING: The script evaluate-cli is installed in '/usr/local/python/3.12.1/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [1]:
# Import required libraries
import json
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Transformers and PyTorch
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import Dataset

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

print("\n✓ Libraries imported successfully")

Using device: cpu

✓ Libraries imported successfully


## 1. Configuration

In [7]:
# Paths
BASE_DIR = Path(r'/workspaces/sem_practise')
DATA_DIR = BASE_DIR / 'data' 
MODEL_DIR = BASE_DIR / 'models' / 'news_classifier'
RESULTS_DIR = BASE_DIR / 'results'

# Create directories
MODEL_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# Model configuration
MODEL_NAME = "FacebookAI/xlm-roberta-base"  # Pretrained multilingual model
MAX_LENGTH = 512
BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_EPOCHS = 3

print(f"Model: {MODEL_NAME}")
print(f"Max sequence length: {MAX_LENGTH}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Number of epochs: {NUM_EPOCHS}")

Model: FacebookAI/xlm-roberta-base
Max sequence length: 512
Batch size: 8
Learning rate: 2e-05
Number of epochs: 3


## 2. Load Data

In [8]:
# Load preprocessed data
with open(DATA_DIR / 'train_data.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)

with open(DATA_DIR / 'test_data.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)

print(f"✓ Loaded {len(train_data)} training samples")
print(f"✓ Loaded {len(test_data)} test samples")

# Convert to DataFrames for easier handling
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

print("\nSample data:")
train_df.head()

✓ Loaded 1431 training samples
✓ Loaded 358 test samples

Sample data:


,text,category
0,"३० पुस, काठमाडौं । दोर्दी खोला जलविद्मुत कम्पन...",bank
1,"के तपाईंलाई थाहा छ ? हाथ, खुट्टाका नङहरुको रंग...",health
2,प्रधानमन्त्री केपी शर्मा ओली थ्री नेसन्स कपको ...,sports
3,"राजविराज, २४ चैत । सप्तरीको मलेकपुर गाउँ विकास...",politic
4,सेञ्चुरीमा गत असोज १ गतेदेखि प्रमूख कार्यकारी ...,bank


In [9]:
# Create label mappings
categories = sorted(train_df['category'].unique())
label2id = {label: idx for idx, label in enumerate(categories)}
id2label = {idx: label for label, idx in label2id.items()}

print(f"Number of categories: {len(categories)}")
print("\nCategory mappings:")
for label, idx in label2id.items():
    print(f"  {idx}: {label}")

# Add numeric labels
train_df['label'] = train_df['category'].map(label2id)
test_df['label'] = test_df['category'].map(label2id)

# Save label mappings
with open(MODEL_DIR / 'label_mappings.json', 'w', encoding='utf-8') as f:
    json.dump({'label2id': label2id, 'id2label': id2label}, f, ensure_ascii=False, indent=2)

print(f"\n✓ Label mappings saved to {MODEL_DIR / 'label_mappings.json'}")

Number of categories: 13

Category mappings:
  0: Agriculture
  1: automobiles
  2: bank
  3: business
  4: economy
  5: education
  6: entertainment
  7: health
  8: politic
  9: sports
  10: technology
  11: tourism
  12: world

✓ Label mappings saved to /workspaces/sem_practise/models/news_classifier/label_mappings.json


## 3. Load Pretrained Model and Tokenizer

In [10]:
# Load tokenizer
print(f"Loading tokenizer: {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print("✓ Tokenizer loaded")

# Load pretrained model for sequence classification
print(f"\nLoading pretrained model: {MODEL_NAME}...")
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(categories),
    id2label=id2label,
    label2id=label2id
)
model.to(device)
print("✓ Model loaded and moved to device")

# Display model info
print(f"\nModel parameters: {model.num_parameters():,}")

Loading tokenizer: FacebookAI/xlm-roberta-base...
✓ Tokenizer loaded

Loading pretrained model: FacebookAI/xlm-roberta-base...


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✓ Model loaded and moved to device

Model parameters: 278,053,645


## 4. Tokenize Data

In [ ]:
def tokenize_function(examples):
    """Tokenize text data"""
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=MAX_LENGTH
    )

# Convert to HuggingFace Dataset format
train_dataset = Dataset.from_pandas(train_df[['text', 'label']])
test_dataset = Dataset.from_pandas(test_df[['text', 'label']])

print("Tokenizing training data...")
train_dataset = train_dataset.map(tokenize_function, batched=True)

print("Tokenizing test data...")
test_dataset = test_dataset.map(tokenize_function, batched=True)

print("\n✓ Tokenization complete")
print(f"Train dataset: {train_dataset}")
print(f"Test dataset: {test_dataset}")

## 5. Training Configuration

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir=str(MODEL_DIR / 'checkpoints'),
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    logging_dir=str(MODEL_DIR / 'logs'),
    logging_steps=10,
    warmup_steps=100,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),  # Use mixed precision if GPU available
)

print("Training configuration:")
print(f"  Output directory: {training_args.output_dir}")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Mixed precision (FP16): {training_args.fp16}")

In [ ]:
# Define metrics
def compute_metrics(eval_pred):
    """Compute accuracy and other metrics"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(labels, predictions)
    return {'accuracy': accuracy}

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

print("✓ Metrics and data collator configured")

## 6. Fine-tune Model

In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("✓ Trainer initialized")
print("\nStarting fine-tuning...\n")

# Train the model
train_result = trainer.train()

print("\n✓ Training complete!")
print(f"Training loss: {train_result.training_loss:.4f}")

## 7. Evaluate Model

In [ ]:
# Evaluate on test set
print("Evaluating model on test set...")
eval_results = trainer.evaluate()

print("\nEvaluation Results:")
for key, value in eval_results.items():
    print(f"  {key}: {value:.4f}")

In [ ]:
# Get predictions
predictions = trainer.predict(test_dataset)
pred_labels = np.argmax(predictions.predictions, axis=1)
true_labels = test_df['label'].values

# Classification report
print("\nDetailed Classification Report:")
print("="*80)
report = classification_report(
    true_labels,
    pred_labels,
    target_names=categories,
    digits=4
)
print(report)

# Save classification report
report_dict = classification_report(
    true_labels,
    pred_labels,
    target_names=categories,
    output_dict=True
)

with open(RESULTS_DIR / 'classification_report.json', 'w', encoding='utf-8') as f:
    json.dump(report_dict, f, ensure_ascii=False, indent=2)

print(f"\n✓ Report saved to {RESULTS_DIR / 'classification_report.json'}")

In [ ]:
# Confusion Matrix
cm = confusion_matrix(true_labels, pred_labels)

plt.figure(figsize=(14, 12))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=categories,
    yticklabels=categories,
    cbar_kws={'label': 'Count'}
)
plt.title('Confusion Matrix - News Classification', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Predicted Category', fontsize=12)
plt.ylabel('True Category', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Confusion matrix saved to {RESULTS_DIR / 'confusion_matrix.png'}")

## 8. Save Fine-tuned Model

In [ ]:
# Save the fine-tuned model
print("Saving fine-tuned model...")
trainer.save_model(str(MODEL_DIR / 'final_model'))
tokenizer.save_pretrained(str(MODEL_DIR / 'final_model'))

print(f"✓ Model saved to {MODEL_DIR / 'final_model'}")

# Save training metrics
metrics = {
    'model_name': MODEL_NAME,
    'num_categories': len(categories),
    'train_samples': len(train_df),
    'test_samples': len(test_df),
    'accuracy': float(eval_results['eval_accuracy']),
    'training_loss': float(train_result.training_loss),
    'epochs': NUM_EPOCHS,
    'batch_size': BATCH_SIZE,
    'learning_rate': LEARNING_RATE
}

with open(MODEL_DIR / 'training_metrics.json', 'w', encoding='utf-8') as f:
    json.dump(metrics, f, ensure_ascii=False, indent=2)

print(f"✓ Metrics saved to {MODEL_DIR / 'training_metrics.json'}")

## 9. Test Predictions on Sample Articles

In [ ]:
def predict_category(text, model, tokenizer, id2label, device):
    """
    Predict category for a given text
    """
    # Tokenize
    inputs = tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=MAX_LENGTH,
        padding=True
    ).to(device)
    
    # Predict
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(predictions, dim=-1).item()
        confidence = predictions[0][predicted_class].item()
    
    return id2label[str(predicted_class)], confidence

# Test on random samples
print("Testing predictions on random samples:\n")
print("="*80)

sample_indices = np.random.choice(len(test_df), 5, replace=False)

for idx in sample_indices:
    text = test_df.iloc[idx]['text']
    true_category = test_df.iloc[idx]['category']
    
    predicted_category, confidence = predict_category(text, model, tokenizer, id2label, device)
    
    print(f"\nText (first 150 chars): {text[:150]}...")
    print(f"True Category: {true_category}")
    print(f"Predicted Category: {predicted_category}")
    print(f"Confidence: {confidence:.4f}")
    print(f"Correct: {'✓' if true_category == predicted_category else '✗'}")
    print("-"*80)

## 10. Summary

In [ ]:
print("="*80)
print("NEWS CLASSIFICATION SUMMARY")
print("="*80)
print(f"\n🤖 Model: {MODEL_NAME} (Pretrained)")
print(f"📊 Categories: {len(categories)}")
print(f"📝 Training samples: {len(train_df)}")
print(f"🧪 Test samples: {len(test_df)}")
print(f"\n📈 Performance:")
print(f"  • Accuracy: {metrics['accuracy']:.4f}")
print(f"  • Training loss: {metrics['training_loss']:.4f}")
print(f"\n💾 Saved Files:")
print(f"  • Model: {MODEL_DIR / 'final_model'}")
print(f"  • Label mappings: {MODEL_DIR / 'label_mappings.json'}")
print(f"  • Training metrics: {MODEL_DIR / 'training_metrics.json'}")
print(f"  • Classification report: {RESULTS_DIR / 'classification_report.json'}")
print(f"  • Confusion matrix: {RESULTS_DIR / 'confusion_matrix.png'}")
print("\n✅ News classification completed successfully!")
print("="*80)